### Imports

In [3]:
import argparse
from inspect import currentframe, getframeinfo
from glob import glob
import numpy as np
import os
from os.path import join
import pandas as pd
from pathlib import Path
import pickle
import sys
import nibabel as nib
from nistats.first_level_model import FirstLevelModel
import warnings
from utils.firstlevel_plot_utils import plot_design
from utils.firstlevel_utils import get_first_level_objs, make_first_level_obj, save_first_level_obj

### Parse Arguments
These are not needed for the jupyter notebook, but are used after conversion to a script for production

- conversion command:
  - jupyter nbconvert --to script --execute 1stlevel_analysis.ipynb

In [4]:
parser = argparse.ArgumentParser(description='First Level Entrypoint script')
parser.add_argument('-data_dir', default='/data')
parser.add_argument('-derivatives_dir', default=None)
parser.add_argument('-fmriprep_dir', default=None)
parser.add_argument('-working_dir', default=None)
parser.add_argument('--subject_ids', nargs="+")
parser.add_argument('--tasks', nargs="+", help="Choose from ANT, CCTHot, discountFix,                                     DPX, motorSelectiveStop, stopSignal,                                     stroop, surveyMedley, twoByTwo, WATT3")
parser.add_argument('--rt', action='store_true')
parser.add_argument('--beta', action='store_true')
parser.add_argument('--n_procs', default=16, type=int)
parser.add_argument('--overwrite', action='store_true')
parser.add_argument('--quiet', '-q', action='store_true')
parser.add_argument('--design_matrix', '-dm', action='store_true')
parser.add_argument('--a_comp_cor', action='store_true')

if '-derivatives_dir' in sys.argv or '-h' in sys.argv:
    args = parser.parse_args()
else:
    args = parser.parse_args([])
    args.tasks = ['discountFix',  #aim 2 tasks
                  'motorSelectiveStop']
#     args.tasks = ['discountFix', 'manipulationTask', #aim 2 tasks
#                   'motorSelectiveStop', 'stopSignal']
    
    # args.tasks = ['ANT', 'CCTHot', 'discountFix', 'DPX', #aim 1 tasks
    #               'motorSelectiveStop', 'stopSignal', 
    #               'stroop', 'twoByTwo', 'WATT3']
    
   
    #args.subject_ids = ['3010']
    args.rt=True
    args.a_comp_cor=True
    args.n_procs=1
    args.derivatives_dir = '/data/derivatives/'
    args.data_dir = '/data'
    args.fmriprep_dir = '/data/derivatives/fmriprep/'
    args.design_matrix=False

In [5]:
if not args.quiet:
    def verboseprint(*args, **kwargs):
        print(*args, **kwargs)
else:
    verboseprint = lambda *a, **k: None # do-nothing function

### Initial Setup

Organize paths and set parameters based on arguments

In [6]:
# Set Paths
derivatives_dir = args.derivatives_dir
if args.fmriprep_dir is None:
    fmriprep_dir = join(derivatives_dir, 'fmriprep', 'fmriprep')
else:
    fmriprep_dir = args.fmriprep_dir
data_dir = args.data_dir
first_level_dir = join(derivatives_dir,'1stlevel')
if args.working_dir is None:
    working_dir = join(derivatives_dir, '1stlevel_workingdir')
else:
    working_dir = join(args.working_dir, '1stlevel_workingdir')

# set tasks
if args.tasks is not None:
    tasks = args.tasks
else:
    tasks = ['discountFix', 'manipulationTask', 
                  'motorSelectiveStop', 'stopSignal']
    
    # tasks = ['ANT', 'CCTHot', 'discountFix', 'DPX', 
    #               'motorSelectiveStop', 'stopSignal', 
    #               'stroop', 'twoByTwo', 'WATT3']
    

# list of subject identifiers
if not args.subject_ids:
    subjects = sorted([i.split("-")[-1] for i in glob(os.path.join(args.data_dir, '*')) if 'sub-' in i])
else:
    subjects = args.subject_ids
    
# other arguments
regress_rt = args.rt
beta_series = args.beta
n_procs = args.n_procs
# TR of functional images
TR = .68
a_comp_cor=args.a_comp_cor

In [7]:
first_level_dir

'/data/derivatives/1stlevel'

In [5]:
verboseprint('*'*79)
verboseprint('Tasks: %s\n, Subjects: %s\n, derivatives_dir: %s\n, data_dir: %s' % 
     (tasks, subjects, derivatives_dir, data_dir))
verboseprint('*'*79)

*******************************************************************************
Tasks: ['discountFix', 'motorSelectiveStop']
, Subjects: ['1021', '1025', '1046', '1105', '1108', '1132', '1143', '1189', '1190', '1211', '1237', '1352', '1376', '1399', '1438', '1481', '1517', '1525', '1585', '1636', '1640', '1646', '1652', '1860', '192', '1961', '1976', '2001', '2099', '211', '2130', '2133', '2143', '2199', '2232', '225', '2265', '232', '233', '2577', '2604', '2689', '2696', '275', '2800', '283', '2834', '2839', '300', '3002', '3010', '31', '3225', '3324', '3364', '3464', '3525', '4022', '4081', '4129', '413', '443', '4591', '4592', '472', '479', '4829', '4930', '5064', '5293', '538', '5387', '5469', '5497', '5579', '579', '5906', '595', '6018', '615', '623', '630', '65', '6638', '667', '685', '698', '7124', '725', '731', '772', '811', '857', '885', '889', '897', '902', '91', '931', '96']
, derivatives_dir: /data/derivatives/
, data_dir: /data
*********************************************

# Set up Nodes

### Run analysis

gather the files for each task within each subject


In [6]:
to_run = []
for subject_id in subjects:
    for task in tasks:
        verboseprint('Setting up %s, %s' % (subject_id, task))
        files = get_first_level_objs(subject_id, task, first_level_dir, regress_rt=regress_rt)
        if len(files) == 0 or args.overwrite:
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore",category=DeprecationWarning)
                warnings.filterwarnings("ignore",category=UserWarning)
                subjinfo = make_first_level_obj(subject_id, task, fmriprep_dir, 
                                                data_dir, TR, regress_rt=regress_rt, a_comp_cor=a_comp_cor)
            if subjinfo is not None:
                to_run.append(subjinfo)

Setting up 1021, discountFix
['/data/derivatives/fmriprep/sub-1021/func/sub-1021_task-discountFix_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz']
Setting up 1021, motorSelectiveStop
['/data/derivatives/fmriprep/sub-1021/func/sub-1021_task-motorSelectiveStop_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz']
Setting up 1025, discountFix
['/data/derivatives/fmriprep/sub-1025/func/sub-1025_task-discountFix_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz']
Setting up 1025, motorSelectiveStop
['/data/derivatives/fmriprep/sub-1025/func/sub-1025_task-motorSelectiveStop_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz']


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

### Run model fit

generate the glm and fit the timeseries data to it

In [ ]:
for subjinfo in to_run:
    verboseprint(subjinfo.ID)
    verboseprint('** fitting model')
    fmri_glm = FirstLevelModel(TR, 
                           subject_label = subjinfo.ID,
                           mask=subjinfo.mask,
                           noise_model='ar1',
                           standardize=False, 
                           hrf_model='spm',
                           drift_model='cosine',
                           period_cut=80,
                           n_jobs=1
                          )
    
    if args.design_matrix:
        verboseprint('** saving')
        save_first_level_obj(subjinfo, first_level_dir, False)
        subjinfo.export_design(first_level_dir)
        subjinfo.export_events(first_level_dir)
    else:
        out = fmri_glm.fit(subjinfo.func, design_matrices=subjinfo.design)
        subjinfo.fit_model = out

        verboseprint('** saving')
        save_first_level_obj(subjinfo, first_level_dir, True)
        subjinfo.export_design(first_level_dir)
        subjinfo.export_events(first_level_dir)
